In [2]:
import numpy as np 
import sys
import os
import re
def set_path():
    if sys.platform == 'darwin':
        print("Current system is macOS")
        main_fold_path = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction'
    elif sys.platform == 'linux':
        print("Current system is Ubuntu")
        main_fold_path = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning'
    else:
        print("Current system is neither macOS nor Ubuntu")
    os.chdir(main_fold_path)
    
set_path()    


import pandas as pd 
def get_best_acc_epoch(csv_file):
    df = pd.read_csv(csv_file)
    index_of_best_acc = df['val_accuracy'].argmax()
    return index_of_best_acc

import glob 

def delete_files_starting_with(directory, prefix):
    pattern = os.path.join(directory, '*' + prefix + '*') 
    files_to_delete = glob.glob(pattern)
    for file in files_to_delete:
        try: 
            os.remove(file)
        except Exception as e:
            print(e)
        

model_name = 'cnn_transformer'
config_name = 'skf_v0mdd_classification_jamba'
dataset = 'diagnosis_406_less_than_threshold'
model_config_dict = {
    # 'jamba_multimodal':[ f'eval{i}mdd_classification_jamba' for i in range(2)],
    # 'cnn_transformer': ['skf_v0mdd_classification_jamba'],
    'jamba': ['eval1mdd_classification_jamba_subject_406',
              'eval2mdd_classification_jamba_subject_406'],
    # 'jamba': ['eval1mdd_classification_jamba_subject_483',
    #           'eval2mdd_classification_jamba_subject_483'],
    # 'jamba': np.flip(['skf_v0mdd_classification_jamba', 
    #           'skf_v1_2layersmdd_classification_jamba', 
    #           'skf_v1_1layers_64_modelstatesmdd_classification_jamba',
    #           'skf_v2_sin_lrmdd_classification_jamba',
    #           'skf_v4_1e5_lrbeginmdd_classification_jamba',
    #           'skf_v5_no_mean_premdd_classification_jamba',
    #           'skf_v6_3layers_256_input_dimsmdd_classification_jamba',
    #           'skf_v7_3layers_256dims_9blocksmdd_classification_jamba',
    #           'skf_v8_2layersmdd_classification_jamba',
    #           'skf_v9_4layersmdd_classification_jamba',
    #           'skf_s0mdd_classification_jamba',
    #           'skf_s1_conv1d_longmdd_classification_jamba',
    #           'skf_s2_conv1d_1000epochs_1e9lrmdd_classification_jamba',
    #           'skf_s3_1000epochs_1e6lrmdd_classification_jamba',
    #           'skf_s4_1e-5lrmdd_classification_jamba',
    #           'skf_s5mdd_classification_jamba',
    #           'skf_s6_replicationmdd_classification_jamba',
    #           'skf_s7_replicat_5000warmupmdd_classification_jamba',
    #           'skf_s8_valaccmdd_classification_jamba',
    #           'skf_s9_valloss_50patiencesmdd_classification_jamba',
    #           'skf_s10_300patiencesmdd_classification_jamba']),
    # 'mamba': ['skf_v0mdd_classification_mamba']
}
validation_name = 'SKF_holdout/stratified_nested_10_CV_fold'
num_of_k_fold = 10

# reading the last index of result
# def find_path_acc(path):
#     with open(path, 'r') as f:
#         acc = f.read()
#     acc = acc.split('\n')[-2]
#     acc = re.findall(r'accuracy: (\d+\.\d+)', acc)[0]
    
#     return np.float(acc)

def find_path_metrics(path):
    with open(path, 'r') as f:
        content = f.read()

    accuracy = re.findall(r'accuracy: (\d+\.\d+)', content)[0]
    sensitivity = re.findall(r'sensitivity: (\d+\.\d+)', content)[0]
    specificity = re.findall(r'specificity: (\d+\.\d+)', content)[0]
    auc = re.findall(r'AUC: (\d+\.\d+)', content)[0]

    # Convert the extracted strings to floats
    accuracy = float(accuracy)
    sensitivity = float(sensitivity)
    specificity = float(specificity)
    auc = float(auc)
    
    return accuracy, sensitivity, specificity, auc
from utils.fnirs_utils import print_md_table_val_test_AUC
def read_model_config_result(model_name, config_name, validation_name='SKF_holdout/stratified_nested_10_CV_fold', num_of_k_fold=10):
    test_metrics = []
    val_metrics = []
    index_best = []
    for k in range(num_of_k_fold):
        path = f'results/{model_name}/{dataset}/{config_name}/{validation_name}-{k}' 
        fold_test_acc = find_path_metrics(path + '/test_acc.txt')
        fold_val_acc = find_path_metrics(path + '/val_acc.txt')
        fold_index_best = get_best_acc_epoch(path + '/history.csv')
        index_best.append(fold_index_best)
        test_metrics.append(fold_test_acc)
        val_metrics.append(fold_val_acc)

        # delete checkpoint files, in case the space is not enough
        delete_files_starting_with(path, 'checkpoint')
    head_print = True if config_name[:7] == 'skf_s10' else False
    print_md_table_val_test_AUC(config_name[:7], np.mean(test_metrics, axis=0), np.mean(val_metrics,axis=0), print_table_header=head_print, already_balanced_accuracy=False)
    # print(val_metrics)
    # print('model_name:', model_name, config_name)
    # print('test_acc:', np.mean(test_metrics))
    # print('val_acc:', np.mean(val_metrics))
    # print('index_best:', np.mean(index_best))


for model_name, config_names in model_config_dict.items():
    for config_name in config_names:
        read_model_config_result(model_name, config_name, validation_name, num_of_k_fold)

Current system is Ubuntu
| eval1md   | 59.8090  | 46.8181  | 72.8000  | 59.8092  | 73.2272  | 66.0000  | 80.4545  | 73.2274  |
| eval2md   | 62.5636  | 52.7273  | 72.4000  | 62.5637  | 75.3637  | 73.0000  | 77.7274  | 75.3637  |


: 

NameError: name 'val_metrics' is not defined

In [15]:
# read the best acc and its corresponding epoch 
import pandas as pd 
def get_best_acc_epoch(csv_file):
    df = pd.read_csv(csv_file)
    index_of_best_acc = df['val_accuracy'].argmax()
    return index_of_best_acc
# csv_file = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/results/jamba/diagnosis514/skf_s2_conv1d_1000epochs_1e9lrmdd_classification_jamba/SKF_holdout/stratified_nested_10_CV_fold-9/history.csv'
# df = pd.read_csv(csv_file)
# index_of_best_acc = df['val_accuracy'].argmax()
# index_of_best_acc